In [ ]:
# Scope Clock Vector Data Generator
#
# 아나로그 오실로스코프를 이용한 시계 입니다.
# 시계얼굴, 시간, 분, 초의 이미지에서 윤곽선을 추출하여 데이터로 저장합니다.
# 이미지의 크기는 4096 x 4096 Pixel 이 최대 크기 입니다.

FaceFileNames   = [                                                               # Clock Face Images
                   'BatmanF1.png',
                   'BatmanF2.png',
                   'BatmanF3.png',
                   'BatmanF4.png',
                   'BatmanF5.png',
                   'BatmanF6.png',
                   'BatmanF7.png',
                   'BatmanF8.png',
                  ]
face_hold_second = 5                                                            # Clock Face 유지 시간(초 단위)

SecondFileNames = [                                                             # Clock Hand Second Images
                   'BatManS1.png',
                   'BatManS2.png',
                   'BatManS3.png',
                   'BatManS4.png',
                  ]
SecondShiftPoint = 0

MinuteFileNames = [                                                             # Clock Hand Minute Images
                   'BatManM1.png',
                   'BatManM2.png',
                   'BatManM1.png',
                   'BatManM3.png'
                  ]
MinuteShiftPoint = 160                                                          # 평행 회전 모드, 160 Pixel 기준

HourFileNames   = [                                                             # Clock Hand Hour Images
                   'BatManH1.png',
                   'BatManH2.png',
                   'BatManH3.png',
                  ]
HourShiftPoint = 0

#===============================================================================
import cv2
import numpy as np
import math
from google.colab.patches import cv2_imshow

#-------------------------------------------------------------------------------
def contourPoint(c, sizeY, sizeX, skip):
  "컨투어의 좌표점을 구합니다."
  px = []
  py = []

  for i, point in enumerate(c):
    if isinstance(point[0], (list, np.ndarray)): # Handle [[x, y]] format
      x, y = point[0]
      y = (sizeY-1)-y
    else: # Handle [x, y] format
      x, y = point
      y = (sizeY-1)-y

    if i%skip==0:
      px.append(x)                ###################
      py.append(y)                ###################

  return px, py, len(px)

#-------------------------------------------------------------------------------
def extract_contours(imgF, step, D=1, P=0):
  '''
  imgF: 윤곽선(Contour) 추출할 이미지
  step: 건너 뛰기 픽셀 개수
  D   : 회전 이미지 생성 개수
  P   : 위쪽으로 부터의 Pixel 수 이며 평행회전 이동 중심거리
  '''
  print('//',imgF, end=' ')
  image = cv2.imread(imgF)

  xd = []; yd = []

  XY = ''
  xy = []
  Count = 0

  # 이미지를 그레이스케일로 변환합니다. findContours는 단일 채널 이미지를 요구합니다.
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 이진화합니다. 이제 gray_image (단일 채널)에 대해 threshold를 적용합니다.
  ret, thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

  # 윤곽선을 찾습니다.
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

  # 찾은 윤곽선을 원본 컬러 이미지에 그립니다.
  #-----------------------------------------------------------------------------
  for i in range(1, len(contours)):                                             # contour의 0 번은 외곽 사각형 이므로 무시합니다.
    cv2.drawContours(image, contours, i, (0, 255, 0), 3)
    xx, yy, cc = contourPoint(contours[i], image.shape[0], image.shape[1], step)# Contour, Y, X, skip Number
    xd = xd+xx; yd = yd+yy
    Count = Count+cc
  #-----------------------------------------------------------------------------
  cv2.rectangle(image,(0,0),(image.shape[1]-1,image.shape[0]-1),(0,0,255),1)
  #params = [cv2.IMWRITE_PNG_COMPRESSION, 9]
  cv2.imwrite('_'+imgF, image)
  #=============================================================================
  XY= ''; CC = 0

  if D <= 1:
    for i in range(Count):
      x = xd[i]                                                                 # X
      y = yd[i]                                                                 # Y
      XY += f'0x{x:03X}, '
      XY += f'0x{y:03X}, '

  elif (D > 1):
    cx = image.shape[1]//2
    cy = image.shape[0]//2

    for minute in range(D):
      p = (90-minute * 360/D)%360   # 0 분= 90 도, 15 분=0도, 30 분=270 도, 45분 = 180도
      angle_radians = math.radians(p)
      yp = int((cy-P) * math.sin(angle_radians)) - (cy-P)
      xp = int((cy-P) * math.cos(angle_radians))

      for i in range(Count):
        if(P==0):
          y = cy - yd[i]                                                        # image.shape[0]//2  Y
          x = xd[i] - cx                                                        # image.shape[1]//2  X
          y_prime = int(y * math.sin(angle_radians) - x * math.cos(angle_radians))
          x_prime = -1*int(x * math.sin(angle_radians) + y * math.cos(angle_radians))
          y = cy - y_prime                                                      # image.shape[0]//2  Y
          x = x_prime + cx                                                      # image.shape[1]//2
        else:
          y = yd[i]+yp                                                          # Y
          x = xd[i]+xp                                                          # X
        x = abs(x)
        if x > image.shape[1]:
          x = image.shape[1] - (x-image.shape[1])
        y = abs(y)
        if y > image.shape[0]:
          y = image.shape[0] - (y-image.shape[0])
        XY += f'0x{x:03X}, '
        XY += f'0x{y:03X}, '

  CC = Count*D

  return XY, CC

################################################################################
XYOUT = ''; COUT = ''; CCN = [0,]; CCNT = '0, '

for imgFile in FaceFileNames:
  T, C = extract_contours(imgFile, 10)  # read image file name, skip, write file name
  A = CCN[-1]+C
  CCN.append(A)
  CCNT = CCNT + str(A) + ', '

  XYOUT = XYOUT + T
  COUT = COUT + str(C) + ', '

FXYOUTP = 'const PROGMEM uint16_t '+ 'clockFace' + '[] = {' + XYOUT + '};'
FCOUTP = 'const PROGMEM uint16_t '+ 'clockFaceCount' + '[] = {' + COUT + '};'
FDOUT = 'const PROGMEM uint32_t '+ 'clockFaceCountOffset' + '[] = {' + CCNT + '};'
FHOLD = f'#define FACE_HOLD_SECOND {face_hold_second} '
print(); print(FXYOUTP); print(FCOUTP); print(FDOUT); print(FHOLD); print()


#===============================================================================
XYOUT = ''; COUT = ''; CCN = [0,]; CCNT = '0, '

for imgFile in SecondFileNames:
  T, C = extract_contours(imgFile, 10, 60, SecondShiftPoint)  # read image file name, skip,
  A = CCN[-1]+C
  CCN.append(A)
  CCNT = CCNT + str(A) + ', '

  XYOUT = XYOUT + T
  COUT = COUT + str(C) + ', '

SXYOUTP = 'const PROGMEM uint16_t '+ 'clockHandSecond' + '[] = {' + XYOUT + '};'
SCOUTP = 'const PROGMEM uint16_t '+ 'clockSecondCount' + '[] = {' + COUT + '};'
SDOUT = 'const PROGMEM uint32_t '+ 'clockSecondCountOffset' + '[] = {' + CCNT + '};'

print(); print(SXYOUTP); print(SCOUTP); print(SDOUT); print()
#===============================================================================
XYOUT = ''; COUT = ''; CCN = [0,]; CCNT = '0, '

for imgFile in MinuteFileNames:
  T, C = extract_contours(imgFile, 10, 60, MinuteShiftPoint)  # read image file name, skip,
  A = CCN[-1]+C
  CCN.append(A)
  CCNT = CCNT + str(A) + ', '

  XYOUT = XYOUT + T
  COUT = COUT + str(C) + ', '

MXYOUTP = 'const PROGMEM uint16_t '+ 'clockHandMinute' + '[] = {' + XYOUT + '};'
MCOUTP = 'const PROGMEM uint16_t '+ 'clockMinuteCount' + '[] = {' + COUT + '};'
MDOUT = 'const PROGMEM uint32_t '+ 'clockMinuteCountOffset' + '[] = {' + CCNT + '};'

print(); print(MXYOUTP); print(MCOUTP); print(MDOUT); print()
#===============================================================================
XYOUT = ''; COUT = ''; CCN = [0,]; CCNT = '0, '

for imgFile in HourFileNames:
  T, C = extract_contours(imgFile, 10, 60, HourShiftPoint)  # read image file name, skip,
  A = CCN[-1]+C
  CCN.append(A)
  CCNT = CCNT + str(A) + ', '

  XYOUT = XYOUT + T
  COUT = COUT + str(C) + ', '

HXYOUTP = 'const PROGMEM uint16_t '+ 'clockHandHour' + '[] = {' + XYOUT + '};'
HCOUTP = 'const PROGMEM uint16_t '+ 'clockHourCount' + '[] = {' + COUT + '};'
HDOUT = 'const PROGMEM uint32_t '+ 'clockHourCountOffset' + '[] = {' + CCNT + '};'

print(); print(HXYOUTP); print(HCOUTP); print(HDOUT); print()

f = open('clockData.txt', 'w')
f.write(FXYOUTP); f.write('\n'); f.write(FCOUTP); f.write('\n'); f.write(FDOUT); f.write('\n'); f.write(FHOLD); f.write('\n'); f.write('\n')
f.write(SXYOUTP); f.write('\n'); f.write(SCOUTP); f.write('\n'); f.write(SDOUT); f.write('\n'); f.write('\n')
f.write(MXYOUTP); f.write('\n'); f.write(MCOUTP); f.write('\n'); f.write(MDOUT); f.write('\n'); f.write('\n')
f.write(HXYOUTP); f.write('\n'); f.write(HCOUTP); f.write('\n'); f.write(HDOUT); f.write('\n'); f.write('\n')
f.close()
